In [1]:
import os, sys
NotebookDirectory = os.path.split(os.getcwd())[0]
if NotebookDirectory not in sys.path:
    sys.path.append(NotebookDirectory)
import toolbox

In [2]:
SciPyModel = toolbox.sbml.importSBMLFile(toolbox.createSciPyModel())
SciPyModel = toolbox.simulation.writeODEFunction( SciPyModel )
SciPyModel = toolbox.sampling.createNullSpaceFunction(SciPyModel)

First we need to ensure the model contains the desired parameterization. This is done by running the parameter estimation script with a properly formatted input set of data. 

In lieu of this implementation, we will choose a random steady-state parameterization to use in this notebook.

In [3]:
import numpy

SciPyModel.Parameters.MinimumValue = numpy.zeros(
    SciPyModel.Parameters.Quantity)
SciPyModel.Parameters.MaximumValue = numpy.ones(
    SciPyModel.Parameters.Quantity) * 2.
SciPyModel.SimulationData.Sensitivity.Global.NumSamples = 25

SciPyModel = toolbox.sampling.uniformNullSpaceSample(SciPyModel)

SciPyModel.Parameters.Value = SciPyModel.SimulationData.Sensitivity.Global.ParameterSets[
    numpy.random.randint(25), :]

The local sensitivity analysis implementation relies on employing the centered difference method to numerically estimate the value of $\frac{d x_j}{d p_i} \frac{p_i}{x_{j}}$ where $x_j$ is the j-th species and $p_i$ is the i-th parameter.

The centered difference method applies a linear approximation which assumes the model, as a function of parameters, exhibits unimodal continuity.

\begin{equation*}
\frac{d x_j}{d p_i} ~=~ \frac{x_j^+ - x_j^-}{p_i^+ - p_i^-} ~=~ \frac{x_j^+ - x_j^-}{p_i(1+q) - p_i(1-q)} ~=~\frac{x_j^+ - x_j^-}{2 q p_i} 
\end{equation*}

Hence once we normalize this equation to the base simulation we see the equation reduce to the following.

\begin{equation*}
\frac{d x_j}{d p_i}\frac{p_i}{x_j} ~=~ \frac{x_j^+ - x_j^-}{2 q p_i}\frac{p_i}{x_j} ~=~ \frac{x_j^+ - x_j^-}{2 q x_j}
\end{equation*}

Based on the equation above, we see that for each parameter sensitivity there are two simulations that must be run: a "high parameter" and "low parameter" simulation.

In [4]:
# Set Simulation Paraeters
SciPyModel.SimulationData.TimeStart = 0
SciPyModel.SimulationData.TimeEnd = 10
SciPyModel.SimulationData.DataPoints = 10

# Integrate Base Simulation
SciPyModel = toolbox.simulation.integrateODEFunction(SciPyModel)
BaseSimulation = SciPyModel.SimulationData.Deterministic.Data[:, :]

# Create Holder for Base Parameterization
BaseParameters = SciPyModel.Parameters.Value[:]

# Set Local Sensitivity Parameters
SciPyModel.SimulationData.Sensitivity.Local.PercentVary = 0.10

# Initialize Sensitivity Capture Array
SciPyModel.SimulationData.Sensitivity.Local.Data = numpy.empty(
    (SciPyModel.Parameters.Quantity, SciPyModel.SimulationData.DataPoints,
     SciPyModel.Species.Quantity))

for i in range(SciPyModel.Parameters.Quantity):
    # Reset Parameter Values
    SciPyModel.Parameters.Value = BaseParameters

    # High Parameter Simulation
    SciPyModel.Parameters.Value[i] = BaseParameters[i] * (
        1 + SciPyModel.SimulationData.Sensitivity.Local.PercentVary)
    SciPyModel = toolbox.simulation.integrateODEFunction(SciPyModel)
    HiSimulation = SciPyModel.SimulationData.Deterministic.Data[:, :]

    # Low Parameter Simulation
    SciPyModel.Parameters.Value[i] = BaseParameters[i] * (
        1 - SciPyModel.SimulationData.Sensitivity.Local.PercentVary)
    SciPyModel = toolbox.simulation.integrateODEFunction(SciPyModel)
    LoSimulation = SciPyModel.SimulationData.Deterministic.Data[:, :]

    # Calculate Sensitivities
    SciPyModel.SimulationData.Sensitivity.Local.Data[i, :, :] = (
        HiSimulation - LoSimulation) / (2 * BaseSimulation[
            i] * SciPyModel.SimulationData.Sensitivity.Local.PercentVary)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in divide


The pre-written, basic visualization implementation included in the Toolbox can be seen below. This can be called immediately following execution of the local sensitivity analysis function.

In [5]:
# Import required packages
from matplotlib import pyplot as plt
# Close previous figures
plt.close()
# Create new figure of required size
plt.figure(figsize=(15, 5 * SciPyModel.Species.Quantity))
# Pad the subplots with whitespace for clarity
plt.subplots_adjust(hspace=1.0)
# Loop over each species
for ix in range(SciPyModel.Species.Quantity):
    # Select current left-column subplot
    plt.subplot(SciPyModel.Species.Quantity, 2, 2 * ix + 1)
    # Plot local sensitivities over time
    PlotHandles = plt.plot(
        SciPyModel.SimulationData.Deterministic.TimeVector,
        SciPyModel.SimulationData.Sensitivity.Local.Data[:, :, ix].transpose())
    # Title the plot
    plt.title(
        'Normalized Local Sensitivities for ' + SciPyModel.Species.Names[ix])
    # Lable the axis
    plt.xlabel('Time')
    plt.ylabel('Normalized Sensitivity')
    # Turn the grid on
    plt.grid()
    # Add legend to plot below the xaxis
    plt.legend(
        PlotHandles,
        SciPyModel.Parameters.Names,
        bbox_to_anchor=(0., -.225, 1., 0),
        ncol=3,
        mode="expand",
        borderaxespad=0.)
    # Select the right-column plot
    plt.subplot(SciPyModel.Species.Quantity, 2, 2 * ix + 2)
    # Plot the bargraph of sensitivities at ending time
    plt.bar(
        range(SciPyModel.Parameters.Quantity),
        SciPyModel.SimulationData.Sensitivity.Local.Data[:, -1, ix])
    # Change the xticks to parameter names
    plt.xticks(
        range(SciPyModel.Parameters.Quantity),
        SciPyModel.Parameters.Names,
        rotation=40)
    # Turn grid on for clarity
    plt.grid()
    # Title the plots for clarity
    plt.title('Normalized Local Sensitivities for ' +
              SciPyModel.Species.Names[ix] + 'at Time ' +
              str(SciPyModel.SimulationData.Deterministic.TimeVector[-1]))
    # Add ylabel
    plt.ylabel('Normalized Sensitivity')
# Display the plots
plt.show()

Demonstrate usage of the implementations.

In [6]:
SciPyModel = toolbox.simulation.calculateLocalSensitivities( SciPyModel )
toolbox.visuals.localSensitivities(SciPyModel)

NameError: global name 'numpy' is not defined